In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from PIL import Image
import librosa
import librosa.display
import torch
import pickle
from einops import rearrange, repeat
import torch.nn.functional as F

np.set_printoptions(suppress=True)
torch.set_printoptions(sci_mode=False)

## 生成测试数据索引

In [36]:
test_index=np.sort(np.random.choice(np.arange(0,150),11,replace=False))
print(test_index)

[ 17  47  62  63  99 122 131 134 137 145 147]


In [37]:
data={}
data['test']=test_index
data

{'test': array([ 17,  47,  62,  63,  99, 122, 131, 134, 137, 145, 147])}

In [38]:
with open('./model_data/index.data','wb') as f:
    pickle.dump(data,f)

## 数据增强

In [ ]:
def add_noise(x, w):
    # 加入高斯白噪音
    output = x + w * np.random.normal(loc=0, scale=1, size=len(x))
    return output.astype(np.float32)

In [ ]:
def stretch(x,sr,rate):
    return librosa.effects.time_stretch(x, rate=rate)

In [ ]:
def roll(x):
    return np.asfortranarray(np.flip(x))

In [ ]:
def stft_transform(y,sr):
    #进行stft变换
    nfft=int(sr*0.7)
    data = librosa.amplitude_to_db(np.abs(librosa.stft(y,n_fft=nfft,hop_length=int(nfft/30))), ref=np.max)
    fig, ax = plt.subplots(1,1)
    librosa.display.specshow(data, sr=sr, x_axis='time', y_axis='linear')
    plt.show()
    return data[12500:]

In [ ]:
actions = os.listdir('./wav/src_wav')
for action in actions:
    folder='./wav/src_wav/'+action+'/'
    print(action)
    for filename in os.listdir(folder):
        i=int(filename[:-4])
        y, sr = librosa.load(folder+filename, sr=None,mono=False)
        ya1 = add_noise(x=y, w=3e-4)
        ya2 = add_noise(x=y, w=3e-4)
        yt1=stretch(x=y,sr=sr,rate=np.random.choice([0.6,0.7,0.8]))
        yt2=stretch(x=y,sr=sr,rate=np.random.choice([1.2,1.3,1.4]))
        librosa.output.write_wav(folder+str(30*1+i)+'.wav',ya1,sr)
        librosa.output.write_wav(folder+str(30*2+i)+'.wav',ya2,sr)
        librosa.output.write_wav(folder+str(30*3+i)+'.wav',yt1,sr)
        librosa.output.write_wav(folder+str(30*4+i)+'.wav',yt2,sr)
        print(folder+filename+' OK')

## 生成高频段样本（用于Transformer）

In [ ]:
def read_wav2(path):
    #读取wav文件
    y, sr = librosa.load(path, sr=48000,mono=False)
    return y,sr

In [ ]:
def stft_transform(y,sr):
    #进行stft变换
    nfft=int(sr*0.7)
    data = librosa.amplitude_to_db(np.abs(librosa.stft(y,n_fft=nfft,hop_length=int(nfft/30))), ref=np.max)
    return data[12500:]

In [ ]:
with open('./model_data/index.data','rb') as f:
    temp=pickle.load(f)
q=temp['query']
k=temp['key']
test=temp['test']

In [ ]:
actions = os.listdir('./model/high_wav')
train_labels=[]
train_stfts=[]
test_labels=[]
test_stfts=[]
for action in actions:
    folder='./wav/high_wav/'+action
    label=int(action[6:])
    print(label)
    for filename in os.listdir(folder):
        i=int(filename[:-4])
        wavaData, framerate = read_wav2(folder+'/'+filename)
        path='./model_data/train_high_data/'+action+'/'+filename[:len(filename)-4]+'.jpg'
        res=stft_transform(wavaData, framerate)
        res=torch.tensor(res)
        if i not in test:
            train_stfts.append(res)
            train_labels.append(label)
        else:
            print(i)
            test_stfts.append(res)
            test_labels.append(label)

In [ ]:
train={}
train['labels']=train_labels
train['stfts']=train_stfts
with open('./model_data/train_high.data','wb') as f:
    pickle.dump(train,f)

In [ ]:
test={}
test['labels']=test_labels
test['stfts']=test_stfts
with open('./model_data/test_high.data','wb') as f:
    pickle.dump(test,f)

## 生成高频段样本（用于CNN）

In [68]:
with open('./model_data/index.data','rb') as f:
    temp=pickle.load(f)
test=temp['test']

In [2]:
with open('./model_data/test_high_img.data','rb') as f:
    data=pickle.load(f)

In [10]:
test_labels=data['labels'][:88]
test_images=data['images']

In [10]:
def get_pic(path):
    img=Image.open(path)
    img=img.resize((224,224))
    img=np.array(img)
    img=torch.tensor(img)
    img=rearrange(img,'w h c -> c w h')
    img=img.float()
    return img

In [15]:
actions = os.listdir('./model_data/Doppler_pic')
train_labels=[]
train_images=[]
test_labels=data['labels']
test_images=data['images']
for action in actions:
    if '8' not in action:
        continue
    folder='./model_data/Doppler_pic/'+action
    label=int(action[6:])
    print(label)
    for filename in os.listdir(folder):
        i=int(filename[:-4])
        path=folder+'/'+filename
        res=get_pic(path)

        test_images.append(res)
        test_labels.append(label)

8


In [11]:
print(test_labels)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]


In [72]:
train={}
train['labels']=train_labels
train['images']=train_images
with open('./model_data/train_high_img.data','wb') as f:
    pickle.dump(train,f)

In [12]:
test={}
test['labels']=test_labels
test['images']=test_images
with open('./model_data/test_high_img.data','wb') as f:
    pickle.dump(test,f)

d:\python3.7\lib\site-packages\torch\storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


## 对于低频数据提取MFCC特征

In [4]:
with open('./model_data/index.data','rb') as f:
    temp=pickle.load(f)
q=temp['query']
k=temp['key']
test=temp['test']

In [50]:
with open('./model_data/test_low.data','rb') as f:
    data=pickle.load(f)

In [32]:
actions = os.listdir('./wav/low_wav/')
train_labels=[]
train_mfccs=[]
test_labels=list(data['labels'])
test_mfccs=data['mfccs']
for action in actions:
    if '8' not in action:
        continue
    folder='./wav/low_wav/'+action+'/'
    label=int(action[6:])
    print(label)
    for filename in os.listdir(folder):
        i=int(filename[:-4])
        path=folder+filename
        y, sr = librosa.load(path, sr=48000,mono=False)
        mfcc = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=24)
        mfcc=torch.tensor(mfcc,dtype=torch.float32)
        mfcc=F.normalize(mfcc,dim=0)
    
        test_mfccs.append(mfcc)
        test_labels.append(label)

8


In [53]:
for item in test_mfccs:
    print(item.shape)

torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size([469, 24])
torch.Size

In [19]:
train={}
train['labels']=train_labels
train['mfccs']=train_mfccs
with open('./model_data/train_low.data','wb') as f:
    pickle.dump(train,f)

d:\python3.7\lib\site-packages\torch\storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [54]:
test={}
test['labels']=test_labels
test['mfccs']=test_mfccs
with open('./model_data/test_low.data','wb') as f:
    pickle.dump(test,f)

## 生成query和key

In [55]:
with open('./model_data/test_high_img.data','rb') as f:
    data=pickle.load(f)
    labels=np.array(data['labels'])
    images=data['images']
with open('./model_data/test_low.data','rb') as f:
    data=pickle.load(f)
    labels=np.array(data['labels'])
    mfccs=data['mfccs']

In [ ]:
query_images=[]
query_mfccs=[]
key_images=[]
key_mfccs=[]
for cls in range(1,9):
    index=np.argwhere(labels==cls).T[0]
    q_index=np.random.choice(index,10,replace=False)
    q_index.sort()
    key=np.setdiff1d(index, q_index)[0]
    key_images.append(images[key].numpy())
    key_mfccs.append(mfccs[key].numpy())
    print(key)
    print(q_index)
    for i in q_index:
        query_images.append(images[i].numpy())
        query_mfccs.append(mfccs[i].numpy())
query_images=np.array(query_images).reshape(8,10,3,224,224)
query_mfccs=np.array(query_mfccs).reshape(8,10,469,24)
key_images=np.array(key_images)
key_mfccs=np.array(key_mfccs)

In [61]:
query_images=torch.from_numpy(query_images)
query_mfccs=torch.from_numpy(query_mfccs)
key_images=torch.from_numpy(key_images)
key_mfccs=torch.from_numpy(key_mfccs)

In [62]:
print(query_images.shape)
print(query_mfccs.shape)
print(key_images.shape)
print(key_mfccs.shape)

torch.Size([7, 10, 3, 224, 224])
torch.Size([7, 10, 469, 24])
torch.Size([7, 3, 224, 224])
torch.Size([7, 469, 24])


In [63]:
query={}
key={}
query['images']=query_images
query['mfccs']=query_mfccs
key['images']=key_images
key['mfccs']=key_mfccs

In [64]:
with open('./model_data/key.data','wb') as f:
    pickle.dump(key,f)
with open('./model_data/query.data','wb') as f:
    pickle.dump(query,f)